# DICOM RT Tool Tutorial with Open-Access Data

This notebook demonstrates the various functions and utilities available in the Dicom RT tool Python package (https://github.com/brianmanderson/Dicom_RT_and_Images_to_Mask) by Anderson et. al. It serves as supplementary information for the Technical Paper titled: <em>"Simple Python Module for Conversions between DICOM Images and Radiation Therapy Structures, Masks, and Prediction Arrays" </em>. This notebook works through an example of publicly available brain tumor data of T1-w/FLAIR MRI sequences and corresponding RT structure files with multiple segmented regions of interest. Full information of the publicly available brain tumor data used in this notebook can be found at: https://figshare.com/articles/dataset/Data_from_An_Investigation_of_Machine_Learning_Methods_in_Delta-radiomics_Feature_Analysis/9943334. This notebook was written for easy accessibility for beginners to Python programming, medical imaging, and computational analysis. It should take no more than 10-15 minutes to run in it's entirety from scratch. The notebook generates about 10 GB worth of files, so ensure you have adequate space to run it. 

The notebook covers the following topics (click to go to section):
1. [Getting the data](#DATA)
2. [Reading in DICOM and RT struct files and converting to numpy array format](#DICOM)
3. [Saving arrays to nifti format and reloading them](#NIFTI)
4. [Saving and loading numpy array files](#NUMPY)
5. [Calculating radiomic features](#RADIOMICS)
6. [Predictions To RT-Structure Example](#RTSTRUCTURE)

The notebook assumes you have the following nested directory structure after running cells that download necessary data:

In [ ]:
"""
Top-level directory/
├── DICOMRTTool_manuscript.ipynb
├── Example_Data/ <- Generated when you run the cells below
|   ├── Image_Data/ 
|       ├── Structure/ <- These correspond to the Pre-RT scans
│           ├── T1/
|               ├── Patient number/
|                   ├── RT Struc file (.dcm) 
│           ├── T2FLAIR/
|               ├── Patient number/
|                   ├── RT Struc file (.dcm)
|       ├── T1/
|           ├── Post1/
|               ├── Patient number/
|                   ├── DICOM image files (.dcm)
|           ├── Post2/
|               ├── Patient number/
|                   ├── DICOM image files (.dcm)
|           ├── Pre/
|               ├── Patient number/
|                   ├── DICOM image files (.dcm) <- The images we care about
|       ├── T2FLAIR/
|           ├── Post1/
|               ├── Patient number/
|                   ├── DICOM image files (.dcm)
|           ├── Post2/
|               ├── Patient number/
|                   ├── DICOM image files (.dcm)
|           ├── Pre/
|               ├── Patient number/
|                   ├── DICOM image files (.dcm) <- The images we care about
├── Data.zip <- Generated when you run the cells below, downloaded Figshare file
├── Nifti_Data/ <- Generated when you run the cells below
|   ├──Image.nii
|   ├──Mask.nii
|   ├──MRN_Path_To_Iteration.xlsx
|   ├──Overall_Data_Examples_(iteration)0.nii.gz 
|   ├──Overall_mask_Examples_y(iteration)0.nii.gz 
├── Numpy_Data/ <- Generated when you run the cells below
|   ├──image.npy
|   ├──mask.npy
├── RT_Structures/ <- Generated when you run the cells below
|   ├──RS_Test_UID.dcm
"""

In [4]:
# Load or install the program, %%capture supresses print statements
!pip install DicomRTTool --upgrade
!pip install flywheel-sdk 
from DicomRTTool.ReaderWriter import DicomReaderWriter
import flywheel

You should consider upgrading via the '/Users/cxl037/PycharmProjects/pythonProject1/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/cxl037/PycharmProjects/pythonProject1/venv/bin/python -m pip install --upgrade pip' command.


In [8]:
# importing neccessary libraries 

# file mangagment 
import os 
import zipfile
from six.moves import urllib

# array manipulation and plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# medical image manipulation 
import SimpleITK as sitk
import shutil

In [6]:
fw = flywheel.Client('flywheel.uwhealth.org:0tfO3O6KmTcoy0fVxM')
FW_GROUP = 'baschnagelgroup'
FW_PROJECT = 'SRS Necrosis Project'
image_path = '/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data'
if not os.path.isdir(image_path):
    os.mkdir(image_path)
fw_group = fw.lookup( FW_GROUP )
fw_project = fw_group.projects.find_first('label={}'.format(FW_PROJECT))
fw_subjects = fw_project.subjects.iter()
# print([subject.label for subject in fw_subjects])
# fw_subject = fw_project.subjects.find_first('label="{}"'.format('1244948'))
# print(fw_subject)

/Users/cxl037/PycharmProjects/pythonProject1/venv/lib/python3.9/site-packages/flywheel/flywheel.py:6300: UserWarning: Client version 15.8.0 does not match server version 16.3.2. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


In [ ]:
!pip install --default-timeout=100 future

In [ ]:
!pip install "requests[security]"

In [ ]:
!pip install antspyx

In [ ]:

for fw_subject in fw_subjects:
    if not fw_subject.label.isnumeric():
        continue
    subject_path = os.path.join(image_path, fw_subject.label)
    if not os.path.isdir(subject_path):
        os.mkdir(subject_path)
    for sess in fw_subject.sessions.iter():
        session_path = os.path.join(subject_path, sess.label)
        if not os.path.isdir(session_path):
            os.mkdir(session_path)
        for acq in sess.acquisitions.iter():
            acq_path = os.path.join(session_path, acq.label)
            #for f in acq.files:
                #if f.type=='dicom':
#                     filepath = '/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/flywheel/{}_{}'.format(fw_subject.label,f.name)
#                     if not os.path.exists(filepath):
#                         acq.download_file(f.name, filepath)
#                     z = zipfile.ZipFile(filepath)
#                     z.extractall(acq_path)
    fw_subject = fw_subject.reload()
            

## Part 1: Getting the data. <a name="DATA"></a>

The RT struc files and their corresponding DICOM images can be in the same directory or different directories. Here we show a case where structure files and images are located in different directories. This is a good dataset to work with since its somewhat messy but coherent enough to show power of DICOMRTTool. Many files (pre-RT, post-RT at 2 timepoints) but only pre-RT T1 and FLAIR images have associated RT structure files. Downloading and unzipping the necessary files will take about 10 minutes on most CPUs and takes up about 8 GB of storage. One may visualize these DICOM images using a free commercially available DICOM viewer, such as Radiant (https://www.radiantviewer.com/).

In [ ]:
img_path = "/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/flywheel_20220217_060337.tar"
output_path = r'/Users/cxl037/PycharmProjects/pythonProject1/Example_Data'

In [ ]:
for subdir, dirs, files in os.walk(os.path.join(output_path, 'flywheel')):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith('.zip'):
            z = zipfile.ZipFile(filepath)
            z.extractall('/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data')
            

In [ ]:
def display_slices(image, mask, skip=1):
    """
    Displays a series of slices in z-direction that contains the segmented regions of interest.
    Ensures all contours are displayed in consistent and different colors.
        Parameters:
            image (array-like): Numpy array of image.
            mask (array-like): Numpy array of mask.
            skip (int): Only print every nth slice, i.e. if 3 only print every 3rd slice, default 1.
        Returns:
            None (series of in-line plots).
    """

    slice_locations = np.unique(np.where(mask != 0)[0]) # get indexes for where there is a contour present 
    slice_start = slice_locations[0] # first slice of contour 
    slice_end = slice_locations[len(slice_locations)-1] # last slice of contour
    
    counter = 1
    
    for img_arr, contour_arr in zip(image[slice_start:slice_end+1], mask[slice_start:slice_end+1]): # plot the slices with contours overlayed ontop
        if counter % skip == 0: # if current slice is divisible by desired skip amount 
            masked_contour_arr = np.ma.masked_where(contour_arr == 0, contour_arr)
            plt.imshow(img_arr, cmap='gray', interpolation='none')
            plt.imshow(masked_contour_arr, cmap='cool', interpolation='none', alpha=0.5, vmin = 1, vmax = np.amax(mask)) # vmax is set as total number of contours so same colors can be displayed for each slice
            plt.show()
        counter += 1

## Part 2: Reading in DICOM and RT struct files and converting to numpy array format. <a name="DICOM"></a>

The principal on which this set of tools operates on is based on the DicomReaderWriter object. It is instantiated with the contours of interest (and associations) and can then be used to create numpy arrays of images and masks of the format [slices, width, height].


The following code logic is used to demonstrate searching a path and returning indices for matched structures and images (by UID) for arbitrary directory structures (DICOM image files and RT Struct files not in the same folder). If all necessary structure files are in the same folder as the corresponding images (by UID), one can alternatively use an os.walk through directories of interest and call DicomReaderWriter each time a folder is discovered. For example, I normally use a folder structure MRN -> date of image (pre,mid,post-RT) -> type of scan (MRI, CT, etc.) -> files (DICOM images + RT Struct). However, this approach calls the DicomReaderWriter iteratively, which can be computationally taxing.

In [48]:
DICOM_path = '/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42' # folder where downloaded data was stored
print(DICOM_path)

/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42


In [10]:
for subject in os.listdir(image_path):
    if subject != '.DS_Store':
        subject_path = os.path.join(image_path, subject)
    # iterate through each session in the subject
    for session in os.listdir(subject_path):
        if session != '.DS_Store':
            session_path = os.path.join(subject_path, session)
        for file in os.listdir(session_path):
            if "ADC (10^-6 mm²_s)" in file:
                print(file)
                shutil.rmtree(os.path.join(session_path, file))

This will walk through all of the folders, and using SimpleITK, will separate them based on SeriesInstanceUIDs.

In [49]:
%%time
Dicom_reader = DicomReaderWriter(description='Examples', arg_max=True)
print('Estimated 30 seconds, depending on number of cores present in your computer')

Dicom_reader.walk_through_folders(DICOM_path)


Estimated 30 seconds, depending on number of cores present in your computer


Loading through DICOM files:   0%|                                                                                                                      | 0/26 [00:00<?, ?it/s]

Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/11 - +c COR T1 CUBE VASCLoading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/802 - rCBV (corrected)(color)Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/951 - +C AX FLAIR reformatLoading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/812 - rCBV (corrected)Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/803 - CBF (color)Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/5 - Ax T2starmpgre







Loading through DICOM files:  27%|█████████████████████████████▌                                                                                | 7/26 [00:00<00:01, 18.73it/s]

Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/1050 - +c SAG STEALTH Reformat
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/1 - 3-Plane Loc SSFSE
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/4 - Ax T2 PROPELLER fat
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/1 - Necrosis Project/Turner^Nelson/RTSTRUCT/20200723/1
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/806 - K2 (color)
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/804 - FMT (color)
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/805 - Tmax (color)


...

Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/6 - Ax T1 Bravo 2.4mm
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/21 - 1.2.840.113711.808.22.4540.663025726.26.1373924172.761311
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/1150 - +C AX T1 Vasc REFORMAT 1.4
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/7 - NeuroQuant Sag 3d


Loading through DICOM files:  69%|███████████████████████████████████████████████████████████████████████████▍                                 | 18/26 [00:01<00:00, 11.90it/s]

Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/799 - Perfusion ROIs
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/816 - K2
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/10 - +c Ax T1 Stealth Bravo/Turner^Nelson/MR/20200723/10
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/2 - Sag T1 FLAIR
Loading from /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/9 - +c Sag T2 FLAIR Cube


Loading through DICOM files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:04<00:00,  6.45it/s]

Compiling dictionaries together...
Index 0, description Perfusion ROIs at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/799 - Perfusion ROIs
Index 1, description CBF (color)  at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/803 - CBF (color)
Index 2, description Tmax (color) at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/805 - Tmax (color)
Index 3, description rCBV (corrected) at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/812 - rCBV (corrected)
Index 4, description rCBV (corrected)(color)  at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/802 - rCBV (corrected)(color)
Index 5, description K2 at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/816 - K2
Index 6, description K2 (color) a

In [22]:
all_rois = Dicom_reader.return_rois(print_rois=True)  # Return a list of all rois present, and print them
print(all_rois)

The following ROIs were found
gtv1
ptv1
['gtv1', 'ptv1']


In [11]:
Contour_Names = [roi for roi in all_rois] 
# Associations work as {'variant_name': 'desired_name'}
associations = {roi:roi for roi in all_rois}
Dicom_reader.set_contour_names_and_associations(Contour_Names=Contour_Names, associations=associations)

Lacking ['l precentral gyrus', 'gtv1', 'ptv1'] in index 0, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/799 - Perfusion ROIs. Found []
Lacking ['l precentral gyrus', 'gtv1', 'ptv1'] in index 1, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/803 - CBF (color). Found []
Lacking ['l precentral gyrus', 'gtv1', 'ptv1'] in index 2, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/805 - Tmax (color). Found []
Lacking ['l precentral gyrus', 'gtv1', 'ptv1'] in index 3, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/812 - rCBV (corrected). Found []
Lacking ['l precentral gyrus', 'gtv1', 'ptv1'] in index 4, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/802 - rCBV (corrected)(color). Found []
Lacking ['l pr

As we can see, these ROIs correspond to a variety of structures. In particular, we can see many GBM and glioma structures. Note GBM denotes glioblastoma multiforme (a high grade glioma).

In [ ]:
# Print the locations of all RTs with a certain ROI name, automatically lower cased
Dicom_reader.where_is_ROI(ROIName='l occipital')

In [ ]:
Dicom_reader.which_indexes_have_all_rois()  # Check to see which indexes have all of the rois we want
# Since we haven't defined anything yet, it prompts you to input a list of contour names

In [ ]:
Dicom_reader.which_indexes_lack_all_rois() # Check to see which indexes LACK all of the rois we want
# Since we haven't defined any wanted ROI yet, it will prompt you to input a list of contour names

In [ ]:
def get_subject(project, label, update=True, **kwargs):
    """Get the Subject container if it exists, else create a new Subject container.
    
    Args:
        project (flywheel.Project): A Flywheel Project.
        label (str): The subject label.
        update (bool): If true, update container with key/value passed as kwargs.
        kwargs (dict): Any key/value properties of subject you would like to update.

    Returns:
        (flywheel.Subject): A Flywheel Subject container.
    """
    
    if not label:
        raise ValueError(f'label is required (currently {label})')
        
    subject = project.subjects.find_first('label="{}"'.format(label))
    if not subject:
        raise ValueError(f'subject {label} is not found')
        
    if update and kwargs:
        subject.update(**kwargs)

    if subject:
        subject = subject.reload()

    return subject

In [ ]:
def get_session(subject, label, update=True, **kwargs):
    """Get the Session container if it exists, else create a new Session container.
    
    Args:
        subject (flywheel.Subject): A Flywheel Subject.
        label (str): The session label.
        update (bool): If true, update container with key/value passed as kwargs.        
        kwargs (dict): Any key/value properties of Session you would like to update.

    Returns:
        (flywheel.Session): A flywheel Session container.
    """
    
    if not label:
        raise ValueError(f'label is required (currently {label})')
        
    session = subject.sessions.find_first('label="{}"'.format(label))
    if not session:
        raise ValueError(f'session {label} is not found')
        
    if update and kwargs:
        session.update(**kwargs)

    if session:
        session = session.reload()

    return session

In [ ]:
def get_or_create_acquisition(session, label, update=True, **kwargs):
    """Get the Acquisition container if it exists, else create a new Acquisition container.
    
    Args:
        session (flywheel.Session): A Flywheel Session.
        label (str): The Acquisition label.
        update (bool): If true, update container with key/value passed as kwargs.        
        kwargs (dict): Any key/value properties of Acquisition you would like to update.

    Returns:
        (flywheel.Acquisition): A Flywheel Acquisition container.
    """
    
    if not label:
        raise ValueError(f'label is required (currently {label})')
        
    acquisition = session.acquisitions.find_first('label="{}"'.format(label))
    if not acquisition:
        acquisition = session.add_acquisition(label=label)
        
    if update and kwargs:
        acquisition.update(**kwargs)

    if acquisition:
        acquisition = acquisition.reload()

    return acquisition

In [ ]:
def upload_file_to_acquisition(acquisition, fp, update=True, **kwargs):
    """Upload file to Acquisition container and update info if `update=True`
    
    Args:
        acquisition (flywheel.Acquisition): A Flywheel Acquisition
        fp (Path-like): Path to file to upload
        update (bool): If true, update container with key/value passed as kwargs.        
        kwargs (dict): Any key/value properties of Acquisition you would like to update.        
    """
    basename = os.path.basename(fp)
    if not os.path.isfile(fp):
        raise ValueError(f'{fp} is not file.')
    attempts = 0
    
    while attempts < 5:
        try: 
            acquisition = acquisition.reload()
            if acquisition.get_file(basename):
                print('file already exists')
                return
            else:
                print('uploading')
                acquisition.upload_file(fp)
#                 while not acquistion.get_file(basename):   # to make sure the file is available before performing an update
#                     acquistion = acquistion.reload()
#                     time.sleep(1)
            break
        except Exception as e:
            print('Error exception caught!')
            print(e)
            attempts += 1
            
    if update and kwargs:
        f = acquisition.get_file(basename)
        f.update(**kwargs)

From these ROIs, we will look for those that describe the following regions of interest: tumor (glioblastoma multiforme only) and high-dose area of radiation therapy. 

In [13]:
nifti_path = '/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Nifti_Data'
for roi in all_rois:
    Contour_Names = [roi] 
    # Associations work as {'variant_name': 'desired_name'}
    associations = {roi:roi}
    Dicom_reader.set_contour_names_and_associations(Contour_Names=Contour_Names, associations=associations)
    indexes = Dicom_reader.which_indexes_have_all_rois()
    pt_indx = indexes[0]
    Dicom_reader.set_index(pt_indx)
    Dicom_reader.get_images_and_mask()
    print(Dicom_reader.series_instances_dictionary[pt_indx]['Description'])
    dicom_sitk_handle = Dicom_reader.dicom_handle # SimpleITK image handle
    mask_sitk_handle = Dicom_reader.annotation_handle # SimpleITK mask handle
    
#     image_nifti_path = os.path.join(nifti_path, '{}_{}_Image.nii'.format('1766382', roi))
#     mask_nifti_path = os.path.join(nifti_path, '{}_{}_Mask.nii'.format('1766382', roi))
#     sitk.WriteImage(dicom_sitk_handle, image_nifti_path)
#     sitk.WriteImage(mask_sitk_handle, mask_nifti_path)
    
#     subject = get_subject(fw_project, '2628744')
#     session = get_session(subject, '2011-08-24 06_42_00')
#     acquisition = get_or_create_acquisition(session, 'Extracted ROIs')
#     upload_file_to_acquisition(acquisition, image_nifti_path)
#     upload_file_to_acquisition(acquisition, mask_nifti_path)
    

Lacking ['l precentral gyrus'] in index 0, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/799 - Perfusion ROIs. Found []
Lacking ['l precentral gyrus'] in index 1, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/803 - CBF (color). Found []
Lacking ['l precentral gyrus'] in index 2, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/805 - Tmax (color). Found []
Lacking ['l precentral gyrus'] in index 3, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/812 - rCBV (corrected). Found []
Lacking ['l precentral gyrus'] in index 4, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/802 - rCBV (corrected)(color). Found []
Lacking ['l precentral gyrus'] in index 5, location /Users/cxl037/PycharmProjects/pythonProjec

SAFIRE 2 STEREO_HEAD_RAD_PLAN 
Lacking ['ptv1'] in index 0, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/799 - Perfusion ROIs. Found []
Lacking ['ptv1'] in index 1, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/803 - CBF (color). Found []
Lacking ['ptv1'] in index 2, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/805 - Tmax (color). Found []
Lacking ['ptv1'] in index 3, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/812 - rCBV (corrected). Found []
Lacking ['ptv1'] in index 4, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/802 - rCBV (corrected)(color). Found []
Lacking ['ptv1'] in index 5, location /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_4

In [ ]:
import ants

In [ ]:
firstfile = '/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Nifti_Data/Pinzer_Kaye_L.nii'
secondfile = '/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Nifti_Data/+c_T2_CUBE_FLAIR_Sag_8.nii'
thirdfile = '/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Nifti_Data/+c_Cor_T1_Cube_10.nii'
fixed=ants.image_read(firstfile)
moving=ants.image_read(secondfile)
moving2=ants.image_read(thirdfile)

In [ ]:
ants.plot(fixed, axis = 2)

In [ ]:
ants.plot(moving, axis = 0)

In [ ]:
ants.plot(moving2, axis = 1)

In [ ]:
reg12 = ants.registration(fixed=fixed, moving=moving, type_of_transform='Similarity')
reg13 = ants.registration(fixed=fixed, moving=moving2, type_of_transform='Similarity')
#mytx = reg13[ 'fwdtransforms'] + reg12[ 'fwdtransforms'] 
mytx = reg13[ 'fwdtransforms']

In [ ]:
test = ants.apply_transform(fixed = fixed, moving = )

In [ ]:
test = ants.registration(fixed=fixed, moving=moving, type_of_transform='Similarity')


In [ ]:
test1 = test['warpedmovout']

In [ ]:
ants.plot(mytx, axis = 2)

In [ ]:
Dicom_reader.set_contour_names_and_associations(Contour_Names=Contour_Names, associations=associations)

Note: The module is printing "Found []" because many of the scans (post-1 and post-2 RT) do not have associated structure files. The module recognizes these images exist (unique UIDs) but associated structure files cannot be located for them.

In [ ]:
indexes = Dicom_reader.which_indexes_have_all_rois()  # Check to see which indexes have all of the rois we want, now we can see indexes

In [ ]:
pt_indx = indexes[-1]
Dicom_reader.set_index(pt_indx)  # This index has all the structures, corresponds to pre-RT T1-w image for patient 011
Dicom_reader.get_images_and_mask()  # Load up the images and mask for the requested index

In [ ]:
image = Dicom_reader.ArrayDicom # image array
mask = Dicom_reader.mask # mask array
dicom_sitk_handle = Dicom_reader.dicom_handle # SimpleITK image handle
mask_sitk_handle = Dicom_reader.annotation_handle # SimpleITK mask handle

In [ ]:
n_slices_skip = 2
display_slices(image, mask, skip = n_slices_skip) # visualize that our segmentations were succesfully convereted 

Note: Cyan color denotes tumor while magenta denotes surrounding area of high-dose radiation. Only displaying 7 slices.

## Part 3: Saving arrays to nifti format. <a name="NIFTI"></a>

If you want to use a manual approach, you can view the nifti files easily after running get_images_and_mask(). Saving files as nifti is advisable since spacing information is preserved.

In [ ]:
nifti_path = '/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Nifti_Data' # nifti subfolder 
if not os.path.exists(nifti_path):
    os.makedirs(nifti_path)

In [ ]:
dicom_sitk_handle = Dicom_reader.dicom_handle # SimpleITK image handle
mask_sitk_handle = Dicom_reader.annotation_handle # SimpleITK mask handle
sitk.WriteImage(dicom_sitk_handle, os.path.join(nifti_path, 'Image.nii'))
sitk.WriteImage(mask_sitk_handle, os.path.join(nifti_path, 'Mask.nii'))

One can also use the built in .write_parallel attribute to generate nifti files for all relevant pairs the DicomReaderWriter object has found/generated. In this case there are 9 image/mask pairs for unique UIDs that contain all contours we are interested in. Note a corresponding log excel file in the specified output path. The nifti files are written in the following format: "Overall_Data_{description}_ {iteration}.nii.gz" (image) or "Overall_mask_{description}_ y{iteration}.nii.gz" (mask).

In [ ]:
%%time
%%capture
Dicom_reader.write_parallel(out_path = nifti_path, excel_file = os.path.join(nifti_path,'.','MRN_Path_To_Iteration.xlsx'))

We can now reload the nifti files and disaply them to check that nothing went wrong. You can inspect the other converted files by changing the numerical suffix as per the excel log file ('MRN_Path_To_Iteration.xlsx').

In [ ]:
nifti_image = sitk.ReadImage(os.path.join(nifti_path,'Image.nii')) # reload image
image = sitk.GetArrayFromImage(nifti_image)
nifti_mask = sitk.ReadImage(os.path.join(nifti_path,'Mask.nii')) # reload mask
mask = sitk.GetArrayFromImage(nifti_mask)

In [ ]:
display_slices(image, mask, skip = n_slices_skip) # visualize that our segmentations were succesfully convereted from nifti 

## Part 4: Saving and loading numpy files for later use. <a name="NUMPY"></a>

Finally we can save the numpy arrays themselves to files for later use (so you don't have to reinstantiate the computationally expensive DicomReaderWriter object) and subsequently re-load the numpy arrays.

In [ ]:
numpy_path = '/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Numpy_Data' # go into numpy subfolder 
if not os.path.exists(numpy_path):
    os.makedirs(numpy_path)

In [ ]:
np.save(os.path.join(numpy_path, 'image'), image) # save the arrays
np.save(os.path.join(numpy_path, 'mask'), mask)

In [ ]:
image = np.load(os.path.join(numpy_path,'image.npy')) # load the arrays
mask = np.load(os.path.join(numpy_path,'mask.npy'))

## Part 5: Radiomics Use-case Example. <a name="RADIOMICS"></a>

Here we use the popular open-source radiomics library PyRadiomics (https://pyradiomics.readthedocs.io/en/latest/) to calculate radiomic features for our ROIs. In this case, we only calculate a limited number features from the tumor as an illustrative example. 

In [ ]:
try:
    from radiomics import featureextractor
except:
    !pip install pyradiomics
    from radiomics import featureextractor

In [ ]:
pd.set_option('display.max_columns', None) # show all columns

In [ ]:
%%time
# note: need sitk images (sitk.ReadImage(nifti file)) to plug into PyRadiomics, preserves spacing 

ROI_index = 1 # index for tumor
nifti_mask_tumor = sitk.BinaryThreshold(nifti_mask, lowerThreshold=ROI_index, upperThreshold=ROI_index) # select only ROI of interest

params = {} # can edit in more params as neccessary 
extractor = featureextractor.RadiomicsFeatureExtractor(**params) # instantiate extractor with parameters 
extractor.disableAllFeatures() # in case where only want some features, can delete disable/enable lines if you want deafult
extractor.enableFeatureClassByName('firstorder') 
extractor.enableFeatureClassByName('glcm') 
features = {} # empty dictionary 
features = extractor.execute(nifti_image, nifti_mask_tumor) # unpack results into features dictionary
df = pd.DataFrame({k: [v] for k, v in features.items()}) # put dictionary into a dataframe 

In [ ]:
df # display dataframe to inspect features 

Numerical results for radiomic features shown here are consistent with importing nifti files as image and label map in 3D Slicer (https://www.slicer.org/) and using Radiomics extension (https://www.slicer.org/wiki/Documentation/Nightly/Extensions/Radiomics).

## Part 6: Predictions To RT-Structure Example <a name="RTSTRUCTURE"></a>

Here we will provide a simple example for converting a predicted NumPy array of a square into a Dicom RT-Structure file

In [ ]:
RT_path = os.path.join('Example_Data', 'RT_Structures')
if not os.path.exists(RT_path):
    os.makedirs(RT_path)

First, we will create a fake prediction, it will be the same size as the image NumPy array

In [ ]:
image = Dicom_reader.ArrayDicom

Now, deep learning model typically create segmentations in the format of (z_images, rows, cols, # of classes) 

In [ ]:
def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

In [ ]:
predictions = np.zeros(image.shape + (4,))  # Four classes: background, square, circle, target
predictions.shape
predictions[75:80, 250:350, 100:200, 1] = 1  # Here we are drawing a square
predictions[75:80, 250:350, 300:400, 2] += create_circular_mask(100, 100, center=None, radius=50).astype('int')
predictions[75:80, 100:200, 200:300, 3] += create_circular_mask(100, 100, center=None, radius=50).astype('int')
predictions[75:80, 100:200, 200:300, 3] -= create_circular_mask(100, 100, center=None, radius=33).astype('int')
predictions[75:80, 100:200, 200:300, 3] += create_circular_mask(100, 100, center=None, radius=15).astype('int')

In [ ]:
display_slices(image, np.argmax(predictions, axis=-1), skip = 1) # visualize our square on the image

Convert the NumPy arrays into RT-Structure

In [ ]:
Dicom_reader.prediction_array_to_RT(prediction_array=predictions, output_dir=RT_path,
                                    ROI_Names=['square', 'circle', 'target'])

# Final notes

### I hope that this code has been useful, if you have any suggestions or problems, please open an issue ticket or merge request on the Github: https://github.com/brianmanderson/Dicom_RT_and_Images_to_Mask

#### Thank you!